Dump of ideas:
- for each row, create new target (for supervised learning), indicate if in the next seven days there will be a short squeeze so can measure success somethow
- Will have to lag the features, rolling agregates ??
- 70ish % to train and 30% to check ?
- Tree but with laged features actually ??? 30 days is very short

In [2]:
import pandas as pd
import numpy as np
import requests
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# # URL to the GitHub folder (NOT raw, NOT the file, but the folder)
# api_url = "https://api.github.com/repos/HardyUnine/ML_Project/contents/plan_b"

# # Get list of files in the folder
# files = requests.get(api_url).json()

# # Extract only CSV file URLs
# csv_urls = [f["download_url"] for f in files if f["name"].endswith(".csv")]


In [ ]:
# Testing on just one of the csv 

url = "https://raw.githubusercontent.com/HardyUnine/ML_Project/main/plan_b/AAPL_AppleInc_2021-05-01_to_2021-05-28.csv"
df = pd.read_csv(url)

print(df.head())

          DAY TICKER COMPANY_NAME  TOTAL_SHARES  SIR  RSI     BF       ADV  \
0  2021-05-01   AAPL   Apple Inc.     500000000   25   50  0.100  30000000   
1  2021-05-02   AAPL   Apple Inc.     500000000   25   48  0.097  29500000   
2  2021-05-03   AAPL   Apple Inc.     500000000   25   46  0.094  29000000   
3  2021-05-04   AAPL   Apple Inc.     500000000   25   44  0.091  28500000   
4  2021-05-05   AAPL   Apple Inc.     500000000   25   42  0.088  28000000   

      PUBLIC  PRICE_PER_SHARE  SS  
0  480000000            100.0   0  
1  480000000             98.5   0  
2  480000000             97.0   0  
3  480000000             95.5   0  
4  480000000             94.0   0  


In [ ]:
# We can remove the company name and total shares I think
# ill lag features for 3 days for RSI, SIR, BF ADV and Price per share
# il roll features like 3day average, 3days standard deviation 
# also add percent change

df.drop(["COMPANY_NAME"], axis=1)

,DAY,TICKER,TOTAL_SHARES,SIR,RSI,BF,ADV,PUBLIC,PRICE_PER_SHARE,SS
0,2021-05-01,AAPL,500000000,25,50,0.100,30000000,480000000,100.0,0
1,2021-05-02,AAPL,500000000,25,48,0.097,29500000,480000000,98.5,0
2,2021-05-03,AAPL,500000000,25,46,0.094,29000000,480000000,97.0,0
3,2021-05-04,AAPL,500000000,25,44,0.091,28500000,480000000,95.5,0
4,2021-05-05,AAPL,500000000,25,42,0.088,28000000,480000000,94.0,0
5,2021-05-06,AAPL,500000000,25,40,0.085,27500000,480000000,92.5,0
6,2021-05-07,AAPL,500000000,25,38,0.082,27000000,480000000,91.0,0
7,2021-05-08,AAPL,500000000,25,36,0.079,26500000,480000000,89.5,0
8,2021-05-09,AAPL,500000000,25,34,0.076,26000000,480000000,88.0,0
9,2021-05-10,AAPL,500000000,25,32,0.073,25500000,480000000,86.5,0


In [10]:
columns_to_lag = ['SIR', 'RSI', 'BF', 'ADV', 'PRICE_PER_SHARE']
lags = [1, 2]  # how many previous days you want

for col in columns_to_lag:
    for lag in lags:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

window = 3  # 3-day window

for col in columns_to_lag:
    df[f'{col}_3day_mean'] = df[col].rolling(window=window).mean()
    df[f'{col}_3day_std'] = df[col].rolling(window=window).std()


df.head()

,DAY,TICKER,COMPANY_NAME,TOTAL_SHARES,SIR,RSI,BF,ADV,PUBLIC,PRICE_PER_SHARE,...,SIR_3day_mean,SIR_3day_std,RSI_3day_mean,RSI_3day_std,BF_3day_mean,BF_3day_std,ADV_3day_mean,ADV_3day_std,PRICE_PER_SHARE_3day_mean,PRICE_PER_SHARE_3day_std
0,2021-05-01,AAPL,Apple Inc.,500000000,25,50,0.100,30000000,480000000,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-02,AAPL,Apple Inc.,500000000,25,48,0.097,29500000,480000000,98.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-03,AAPL,Apple Inc.,500000000,25,46,0.094,29000000,480000000,97.0,...,25.0,0.0,48.0,2.0,0.097,0.003,29500000.0,500000.0,98.5,1.5
3,2021-05-04,AAPL,Apple Inc.,500000000,25,44,0.091,28500000,480000000,95.5,...,25.0,0.0,46.0,2.0,0.094,0.003,29000000.0,500000.0,97.0,1.5
4,2021-05-05,AAPL,Apple Inc.,500000000,25,42,0.088,28000000,480000000,94.0,...,25.0,0.0,44.0,2.0,0.091,0.003,28500000.0,500000.0,95.5,1.5


In [ ]:
# Load all CSVs into one dataframe
dfs = []
for url in csv_urls:
    data = pd.read_csv(url)
    data["source_file"] = url.split("/")[-1].replace(".csv", "")  # optional tag
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)